In [24]:
import pandas as pd
import os

# ---
run_prefix = "no_app"   # prefix of output run names
base_climdir = "/home/tykukla/aglime-swap-cdr/scepter/clim-inputs"
clim_general = "cesm2le_monthly_ens1231-001"   # identifier for climate files to use
clim_ts = "1851-1950mean_x100yr"
const_dict = {"duration": 80,
              "dustrate": 0,
              "climatedir": os.path.join(base_climdir, clim_general, clim_ts),
             }
# ---
savefn = "no_app_v0.csv"

In [25]:
# --- read in site inputs
spin_in_path = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"
spin_inputs = "spinup-inputs.csv"
# read in 
df = pd.read_csv(os.path.join(spin_in_path, spin_inputs))
df

,site,spinname,lat,lon,mat,soilmoisture,qrun,tsom,erosion,nitrif,tph,cec,tec,tsoilco2,poro,alpha
0,site_311,site_311,42.5,-91,8.22219,0.282727,0.351361,2.051667,0.001013,1.005952,6.058007,21.10329,20.98031,-1.80371,0.447,2.0
1,site_411,site_411,32.0,-83,18.52789,0.231552,0.243426,2.276667,0.000840,0.831883,5.200242,1.96125,46.91557,-1.61194,0.419,2.0


In [26]:
# define the vars to collect
allvars = ['duration', 'cec', 'dustrate', 'spinrun', 'newrun_id', 'climatedir', 'climatefiles']

In [27]:
def newrun_id_fxn(pref, site, clim_tag):
    return pref + "_" + site + "_" + clim_tag
    

In [28]:
# --- functions to derive inputs
def thisrow(tdf, const_dict, allvars, run_prefix, clim_ts):
    outarr = [] # save row
    thissite = tdf["site"]
    for var in allvars:
        if var == 'spinrun':
            outarr.append(thissite)
        elif var == 'newrun_id':
            outarr.append(newrun_id_fxn(run_prefix, thissite, clim_ts))
        elif var == 'climatefiles':
            outarr.append(thissite)
        else:   # var is defined by const_dict or df      
            tvar = const_dict.get(var)
            if tvar is None:
                tvar = tdf[var]
                outarr.append(tvar)
            else:
                outarr.append(tvar)
    return outarr

In [29]:
# --- loop through sites
# create output pandas df
outdf = pd.DataFrame(columns=allvars)
# loop
for idx in range(len(df)):
    tdf = df.loc[idx]
    thesevars = thisrow(tdf, const_dict, allvars, run_prefix, clim_ts)
    outdf.loc[idx] = thesevars
outdf

,duration,cec,dustrate,spinrun,newrun_id,climatedir,climatefiles
0,80,21.10329,0,site_311,no_app_site_311_1851-1950mean_x100yr,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
1,80,1.96125,0,site_411,no_app_site_411_1851-1950mean_x100yr,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411


In [30]:
outdf['climatedir'][1]

'/home/tykukla/aglime-swap-cdr/scepter/clim-inputs/cesm2le_monthly_ens1231-001/1851-1950mean_x100yr'

In [34]:
# --- save result
savepath_batch = '/home/tykukla/aglime-swap-cdr/scepter/batch-inputs'
outdf.to_csv(os.path.join(savepath_batch, savefn), index=False)